# ***NNDC PARSER TOOL***

This notebook demonstrates usages of the `nndc_parser.py` module and how their implementation can be used to adapt <a href="https://www.nndc.bnl.gov/nudat3/getdataset.jsp?nucleus=37Si">NNDC CSV</a> files for isomeric gamma emission particularly for the `Isomer_DB.sqlite` file accessed in LISE++.

With modificaiton of the module, or custom implementation of functions, this code may be adopted to tables of various formats once it is understood.

The package is imported and function usages are demonstrated throughout the notebook.

In [1]:
import nndc_parser # import module

d:\Users\Hudson\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
nndc_parser.__all__ # all function names

['sqlite_characterize_table',
 'sqlite_to_pandas',
 'parse_isomerID',
 'stretch_CSV',
 'sqlite_entries',
 'input_entries']

#### ***Directory pathing***

Perform command line utilities `pwd` to get directory and run `cd {nndc_csvs folder directory}` to run code.

In [3]:
pwd

'c:\\Users\\hudso\\OneDrive\\working\\IsomerProject\\NNDC_Code'

In [4]:
cd nndc_csvs/

c:\Users\hudso\OneDrive\working\IsomerProject\NNDC_Code\nndc_csvs


#### ***Function Usages***

Below are demonstrations of function utilities internally called within `sqlite_entries` to append the data to a .sqlite database.

Some functions are external, but used together with results from `sqlite_entries` or `stretch_csv` for intermediate steps if the user wishes to have some custom table/implementation of the code.

In [5]:
# read in the table from the dataframe and copy to a Pandas DataFrame
Isom_DB = nndc_parser.sqlite_to_pandas('Isomer_DB_TSTplaying.sqlite', 'Isomers')
Isom_DB

,INDEX_IT,A_IT,Z_IT,E_GAMMA,D_EG,IT_RATIO,D_IT_RATIO,T12,D_T12,LEVEL,...,I_GAMMA,D_IG,M_GAMMA,M_RATIO,D_MRATIO,CONV,D_CONV,SOURCE,NAME,ROW
0,12040142.0,12,4,142.00,2.00,10,0,0.2330,8.00E-03,2251.0,...,None,None,E2,None,None,None,None,Atlas2,DK,1
1,12042251.0,12,4,2251.00,1.00,10,None,0.2330,8.00E-03,2251.0,...,None,None,E0,None,None,None,None,Atlas2,DK,2
2,16070120.0,16,7,120.42,0.12,10,None,5.3060,2.80E-02,120.42,...,None,None,None,None,None,None,None,Atlas2,DK,3
3,18090184.0,18,9,184.00,None,10,None,0.1600,4.00E-03,1121.36,...,100,AP,E2,None,None,None,None,NNDC,OT,4
4,19090087.0,19,9,87.30,None,10,None,0.0893,1.00E-02,197.143,...,0.06,AP,None,None,None,None,None,NNDC,OT,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,214860282.0,214,86,281.50,,10,,0.0037,.0003,2676.1,...,100,1.4,E2,,,,,NNDC,DK,3327
3494,94350203.0,94,35,202.60,,10,,0.5300,1.50E-02,203.0,...,,,"E1,M1,E2",,,,,"PHYSICAL REVIEW C 86, 054319",DK,512
3495,94350088.0,94,35,88.00,,10,,0.5300,1.50E-02,203.0,...,,,,,,,,"PHYSICAL REVIEW C 86, 054319",DK,507
3496,94350116.0,94,35,115.50,,10,,0.5300,1.50E-02,116.0,...,,,,,,,,"PHYSICAL REVIEW C 86, 054319",DK,509


In [6]:
# Characterize table from .sqlite database
columns, recent_guid = nndc_parser.sqlite_characterize_table('Isomer_DB_TSTplaying.sqlite', 'Isomers')
print('Columns in table:',columns)
print('Recent GUID value:',recent_guid)

Columns in table: ['INDEX_IT', 'A_IT', 'Z_IT', 'E_GAMMA', 'D_EG', 'IT_RATIO', 'D_IT_RATIO', 'T12', 'D_T12', 'LEVEL', 'D_LEVEL', 'JPI', 'I_GAMMA', 'D_IG', 'M_GAMMA', 'M_RATIO', 'D_MRATIO', 'CONV', 'D_CONV', 'SOURCE', 'NAME', 'ROW']
Recent GUID value: 3498


In [7]:
# Isomer ID parser for creating INDEX_IT values
Anum, Element, Znum = nndc_parser.parse_isomerID('adoptedLevels37Si.csv')
print('Mass:',Anum,'Element:',Element,'Proton:',Znum)

Mass: 37 Element: Si Proton: 14


In [8]:
# Isomer ID parser with error -- see "Usage with CSV files" for extended explanation
Anum, Element, Znum = nndc_parser.parse_isomerID('gammaDataSi-37.csv')
print('Mass:',Anum,'Element:',Element,'Proton:',Znum)

ValueError: Check filename format (force 'adoptedLevels###SYMBOL.csv' shape)

#### ***Usage with CSV files***

The `.csv` files should be obtained from the NNDC isomer information page linked <a href="https://www.nndc.bnl.gov/nudat3/getdataset.jsp?nucleus=37Si">here</a>.

The file name format is crucial for emission ID splicing and creation of unique ID's based on the `[Mass][Proton][Gamma Emisison]` formatting in `INDEX_IT`.

The data format in the CSV is also crucial as the code works for the NNDC's specific data formatting.

In [9]:
# Simplest implementation of code with .sqlite_entries
entries, expanded_csv =  nndc_parser.sqlite_entries('adoptedLevels119In.csv', user_name = 'HM', db_dir = 'Isomer_DB_TSTplaying.sqlite', subtable = 'Isomers', extra_tables=True)
# Drawn entry format
entries

,INDEX_IT,A_IT,Z_IT,E_GAMMA,D_EG,IT_RATIO,D_IT_RATIO,T12,D_T12,LEVEL,...,I_GAMMA,D_IG,M_GAMMA,M_RATIO,D_MRATIO,CONV,D_CONV,SOURCE,NAME,ROW
0,11949311,119,49,311.39,3,10,NaN,1080000000.0,1800.0,311.37,...,100,NaN,M4,NaN,NaN,NaN,NaN,NNDC,HM,3499
1,11949293,119,49,292.9,1,10,NaN,0.001,LT,604.18,...,100,NaN,"M1,E2",NaN,NaN,NaN,NaN,NNDC,HM,3500
2,1194950,119,49,50.2,1,10,NaN,0.13,0.000002,654.27,...,2.1,2,[E1],NaN,NaN,NaN,NaN,NNDC,HM,3501
3,11949343,119,49,343.0,1,10,NaN,0.13,0.000002,654.27,...,100,9,E1,NaN,NaN,NaN,NaN,NNDC,HM,3502
4,11949721,119,49,720.7,2,10,NaN,0.002,0.000002,720.60,...,100,NaN,NaN,NaN,NaN,NaN,NaN,NNDC,HM,3503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,119491540,119,49,1539.9,3,10,NaN,0.24,0.000003,2564.64,...,55,15,NaN,NaN,NaN,NaN,NaN,NNDC,HM,3605
107,119491623,119,49,1623.0,5,10,NaN,0.24,0.000003,2564.8,...,30,13,NaN,NaN,NaN,NaN,NaN,NNDC,HM,3606
108,119491911,119,49,1911.0,10,10,NaN,0.24,0.000003,2564.8,...,31,19,NaN,NaN,NaN,NaN,NaN,NNDC,HM,3607
109,119491961,119,49,1960.7,3,10,NaN,0.24,0.000003,2564.8,...,100,19,NaN,NaN,NaN,NaN,NaN,NNDC,HM,3608


In [10]:
# Intermediate step with Stretch_CSV
expanded_csv

,E(level)(keV),XREF,Jπ(level),T1/2(level),E(γ)(keV),I(γ),M(γ),Final Levels,Final Jπ,RATIOS,DECAY_PARENT
0,"[311.37, 3]",ABCDE,1/2-,"[18.0, m, 3]","[311.39, 3]","[100, ]",M4,0,9/2+,"[' % IT = 4.4', ' % B- = 95.6']",
1,"[604.18, 7]",ABDE,3/2-,"[1.0, ns, <]","[292.9, 1]","[100, ]","M1,E2",311.37,1/2-,,
2,"[654.27, 7]",AB,(3/2)+,"[130, ns, 15]","[50.2, 1]","[2.1, 2]",[E1],604.18,3/2-,,
3,"[654.27, 7]",AB,(3/2)+,"[130, ns, 15]","[343.0, 1]","[100, 9]",E1,311.37,1/2-,,
4,"[720.60, 11]",BF,(7/2+),"[2.00, ns, 15]","[720.7, 2]","[100, ]",NaN,0,9/2+,,788.26 9
...,...,...,...,...,...,...,...,...,...,...,...
106,"[2564.64, 17]",B,(11/2-),"[240, ns, 25]","[1539.9, 3]","[55, 15]",,1025.02,(11/2+),,2656.9 18
107,"[2564.8, 3]",A,"1/2+,3/2+,5/2+","[240, ns, 25]","[1623.0, 5]","[30, 13]",,941.43,5/2+,,2656.9 18
108,"[2564.8, 3]",A,"1/2+,3/2+,5/2+","[240, ns, 25]","[1911.0, 10]","[31, 19]",,654.27,(3/2)+,,2656.9 18
109,"[2564.8, 3]",A,"1/2+,3/2+,5/2+","[240, ns, 25]","[1960.7, 3]","[100, 19]",,604.18,3/2-,,2656.9 18


In [11]:
# Use pandas dataframe to input entries to .sqlite table
nndc_parser.input_entries(entries, 'Isomer_DB_TSTplaying.sqlite', 'Isomers')

#### ***Information control:***

If testing the package in a virtual environment, here is connection with sqlite3 that lets the user pass SQL queries and modify the database in the case of duplicate information.

In [12]:
# # Delete entries of some username using a connection and sqlite3.
# import sqlite3
# conn = sqlite3.connect('Isomer_DB_TSTplaying.sqlite')
# curs = conn.cursor()
# curs.execute("DELETE FROM Isomers WHERE NAME = 'HM'")
# conn.commit()
# conn.close()